In [ ]:
%pip install requests

***We are scraping posts(links of posts) here and add them into a list in order to investigate further in the next steps.***

In [85]:
import requests
from bs4 import BeautifulSoup

urls = [
    #'https://mitmachen.siegburg.de/angebotslandkarte',
    'https://mitmachen.jena.de/projekts',
    #'https://mitreden.ilzerland.bayern/projekts'
]

def def_42(urls):
    urls_and_data = {}

    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to retrieve the webpage. Status code:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        link_and_content = scraper(soup, url)
        urls_and_data[url] = link_and_content

    return urls_and_data


def scraper(soup, url):
    if 'siegburg' in url:
        return siegburg_data(soup)
    elif 'jena' in url:
        return jena_data(soup)
    elif 'ilzerland' in url:
        return ilzerland_data(soup)
    
    return "No scraper defined for this URL"

def_42(urls)

{'https://mitreden.ilzerland.bayern/nachnutzung-gasthaus-zum-braeu': 'Nachnutzung Gasthaus Zum Bräu: Mitten im Ort soll ein neues SmartesLand-Projekt der Gemeinde Innernzell entstehen.Ein Leerstand, das ehemalige "Gasthaus Zum Bräu", soll im Rahmen des Modellprojekts Smart City wieder mit Leben gefüllt werden.Die Räumlichkeiten in zentraler Lage bieten sich für vielfältige Nutzungen an.Denkbar wäre, dass Vereine, die Räume nutzen können und unterschiedlichste Veranstaltungen wie etwa ein „Dorfkino“, Feiern, Seminare etc. hier stattfinden können.Was ist deine Idee dazu? Nutze die Chance und mache mit vom 4. bis 18. Oktober 2022.Teile deine Idee gerne weiter unten als Vorschlag auf MITREDEN. Wir freuen uns auf deine Idee!Die eingereichten Ideen dienen dem Gemeinderat als Diskussionsgrundlage.', 'https://mitreden.ilzerland.bayern/jugendforum-schoefweg': "Jugendforum Schöfweg: Deine Meinung ist uns wichtig!Ihr habt das Recht mitzureden!Gemeinsam packen wir's an!Du bist die Zukunft!Jugendfo

{'https://mitmachen.jena.de/projekts': {'https://mitmachen.jena.de/matschspielplatz-schlippenstrasse': 'Matschspielplatz Schlippenstraße: In Wenigenjena, konkret auf dem \xa0Spielplatz Schlippenstraße, soll der erste Matschspielplatz in Jena entstehen. Der im Vergleich zu anderen Spielplätzen eher kleinere Platz wurde 1990 errichtet und im Jahr 2005 sind zuletzt die Spielgeräte erneuert worden. Nun soll die Gesamtfläche von ca. 600 m² als Matschspielpatz umgestaltet werden. Der Spielplatz soll das ganze Jahr Gruppen- und Einzelspielmöglichkeiten für Kinder von 0 bis 6 Jahren bieten.Bei der Errichtung von Spielplätzen mit dem Element Wasser gilt es nach den Vorschriften, immer auch eng mit den Gesundheitsämtern zusammenzuarbeiten. Das Spielen darf nicht krank machen, sondern soll allen Spaß (und maximal schmutzige Klammotten) bereiten. Deshalb werden wir in Jena keine stehende Gewässer in Form von Wasserbecken oder ähnlichen Elementen anbieten.Die Planung des Spielplatzes besteht - wie 

In [84]:
def siegburg_data(soup):
    links = []

    a_tags = soup.find_all('a')

    for tag in a_tags:
        href = tag.get('href')
        
        if href and '/proposals/' in href and not 'new' in href:
            link = 'https://mitmachen.siegburg.de' + href
    
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'siegburg')

    return link_and_content


def jena_data(soup):
    links = []

    a_tags = soup.find_all('a',  class_='resource-item--title')

    for tag in a_tags:
        href = tag.get('href')
        
        if href:
            link = 'https://mitmachen.jena.de' + href
            
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'jena')

    return link_and_content

def ilzerland_data(soup):
    links = []

    a_tags = soup.find_all('a',  class_='resources-list--inner')

    for tag in a_tags:
        href = tag.get('href')
        
        if href:
            link = 'https://mitmachen.ilzerland.de' + href
            
            if not link in links:
                links.append(link)
    
    link_and_content = content_scraper(soup, links, 'ilzerland')
    print(link_and_content)
    return link_and_content

In [83]:
def content_scraper(soup, links, identifier):
    link_and_data = {}

    for link in links:
        response = requests.get(link)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            match identifier:
                case 'jena':
                    title = soup.find_all('title')
                    content = soup.find_all('div', class_='flex-layout')

                    text_content = title[0].get_text(strip=True) + ': ' + content[0].get_text(strip=True)
                case 'siegburg':
                    divs = soup.find_all('div', class_='flex-layout')
                    content = divs[1]

                    text_content = content.get_text(strip=True)
                case 'ilzerland':
                    title = soup.find_all('title')
                    print(title)
                    content = soup.find_all('div', class_='custom-page-content')
                    print(content)
                    text_content = title[0].get_text(strip=True) + ': ' + content[0].get_text(strip=True)
                
                case _:
                    text_content = "Wrong identifier"

            link_and_data[link] = text_content

        else:
            message = "Failed to retrieve the webpage. Status code:", response.status_code
            link_and_data[link] = message
    
    return link_and_data